# Machine learning pipelines: Node classification

In [3]:
import os
import json
import numpy as np
import pandas as pd
from graphdatascience import GraphDataScience

Connect to Neo4j

In [4]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_DB = "neo4j"
NEO4J_AUTH = (
    "neo4j",
    "12345678",
)
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH, database=NEO4J_DB)

## Loading the Cora dataset

The CSV files can be found at the following URIs:

In [5]:
CORA_CONTENT = (
    "https://raw.githubusercontent.com/neo4j/graph-data-science/master/test-utils/src/main/resources/cora.content"
)
CORA_CITES = (
    "https://raw.githubusercontent.com/neo4j/graph-data-science/master/test-utils/src/main/resources/cora.cites"
)

Upon loading, we need to perform an additional preprocessing step to convert the `subject` field (which is a string in the dataset) into an integer, because node properties have to be numerical in order to be projected into a graph; 

We also select a number of nodes to be held out to test the model after it has been trained.

In [6]:
SUBJECT_TO_ID = {
    "Neural_Networks": 0,
    "Rule_Learning": 1,
    "Reinforcement_Learning": 2,
    "Probabilistic_Methods": 3,
    "Theory": 4,
    "Genetic_Algorithms": 5,
    "Case_Based": 6,
}

HOLDOUT_NODES = 10

We can now load the CSV files using the `LOAD CSV` Cypher statement and some basic data transformation:

In [ ]:
# Define a string representation of the SUBJECT_TO_ID map using backticks
subject_map = json.dumps(SUBJECT_TO_ID).replace('"', "`")

# Cypher command to load the nodes using `LOAD CSV`, taking care of
# converting the string `subject` field into an integer and
# replacing the node label for the holdout nodes
load_nodes = f"""
    LOAD CSV FROM "{CORA_CONTENT}" AS row
    WITH 
      {subject_map} AS subject_to_id,
      toInteger(row[0]) AS extId, 
      row[1] AS subject, 
      toIntegerList(row[2..]) AS features
    MERGE (p:Paper {{extId: extId, subject: subject_to_id[subject], features: features}})
    WITH p LIMIT {HOLDOUT_NODES}
    REMOVE p:Paper
    SET p:UnclassifiedPaper
"""

# Cypher command to load the relationships using `LOAD CSV`
load_relationships = f"""
    LOAD CSV FROM "{CORA_CITES}" AS row
    MATCH (n), (m) 
    WHERE n.extId = toInteger(row[0]) AND m.extId = toInteger(row[1])
    MERGE (n)-[:CITES]->(m)
"""

# Load nodes and relationships on Neo4j
gds.run_cypher(load_nodes)
gds.run_cypher(load_relationships)

With the data loaded on Neo4j, we can now project a graph including all the nodes and the `CITES` relationship as undirected (and with `SINGLE` aggregation, to skip repeated relationships as a result of adding the inverse direction).

In [ ]:
# Create the projected graph containing both classified and unclassified nodes
G, _ = gds.graph.project(
    "cora-graph",
    {"Paper": {"properties": ["features", "subject"]}, "UnclassifiedPaper": {"properties": ["features"]}},
    {"CITES": {"orientation": "UNDIRECTED", "aggregation": "SINGLE"}},
)

## Pipeline catalog basics

Once the dataset has been loaded, we can define a node classification machine learning pipeline.

In [ ]:
# Create the pipeline
node_pipeline, _ = gds.beta.pipeline.nodeClassification.create("cora-pipeline")

We can check that the pipeline has actually been created with the `list` method:

In [ ]:
# List all pipelines
gds.beta.pipeline.list()

# Alternatively, get the details of a specific pipeline object
gds.beta.pipeline.list(node_pipeline)

,pipelineInfo,pipelineName,pipelineType,creationTime
0,"{'featurePipeline': {'nodePropertySteps': [], ...",cora-pipeline,Node classification training pipeline,2023-05-15T00:57:55.790832600+07:00


## Configuring the pipeline

We can now configure the pipeline. We need to:

1. Select a subset of the available node properties to be used as features for the machine learning model
1. Configure the train/test split and the number of folds for k-fold cross-validation _(optional)_
1. Configure the candidate models for training
1. Configure autotuning _(optional)_
In this example we use Logistic Regression as a candidate model for the training, but other algorithms (such as Random Forest) are available as well. We also set some reasonable starting parameters that can be further tuned according to the needed metrics.

Some hyperparameters such as `penalty` can be single values or ranges. If they are expressed as ranges, autotuning is used to search their best value.

The `configureAutoTuning` method can be used to set the number of model candidates to try. Here we choose 5 to keep the training time short.

In [ ]:
# "Mark" some node properties that will be used as features
node_pipeline.selectFeatures(["features"])

# If needed, change the train/test split ratio and the number of folds
# for k-fold cross-validation
node_pipeline.configureSplit(testFraction=0.2, validationFolds=5)

# Add a model candidate to train
node_pipeline.addLogisticRegression(maxEpochs=200, penalty=(0.0, 0.5))

# Explicit set the number of trials for autotuning (default = 10)
node_pipeline.configureAutoTuning(maxTrials=5)

name                                                     cora-pipeline
nodePropertySteps                                                   []
featureProperties                                           [features]
splitConfig                {'testFraction': 0.2, 'validationFolds': 5}
autoTuningConfig                                      {'maxTrials': 5}
parameterSpace       {'MultilayerPerceptron': [], 'RandomForest': [...
Name: 0, dtype: object

## Training the pipeline

The configured pipeline is now ready to select and train a model. We also run a training estimate, to make sure there are enough resources to run the actual training afterwards.

The Node Classification model supports several evaluation metrics. Here we use the global metric `F1_WEIGHTED`.

In [ ]:
# Estimate the resources needed for training the model
node_pipeline.train_estimate(
    G,
    targetNodeLabels=["Paper"],
    modelName="cora-pipeline-model",
    targetProperty="subject",
    metrics=["F1_WEIGHTED"],
    randomSeed=42,
    concurrency=4,
)

requiredMemory                                     [64 MiB ... 64 MiB]
treeView             Memory Estimation: [64 MiB ... 64 MiB]\r\n|-- ...
mapView              {'components': [{'components': [{'components':...
bytesMin                                                      67130384
bytesMax                                                      67162344
nodeCount                                                         2698
relationshipCount                                                10502
heapPercentageMin                                                  0.1
heapPercentageMax                                                  0.1
Name: 0, dtype: object

In [ ]:
# Perform the actual training
model, stats = node_pipeline.train(
    G,
    targetNodeLabels=["Paper"],
    modelName="cora-pipeline-model",
    targetProperty="subject",
    metrics=["F1_WEIGHTED"],
    randomSeed=42,
    concurrency=4,
)

We can inspect the result of the training, for example to print the evaluation metrics of the trained model.

In [ ]:
# Print F1_WEIGHTED metric
stats["modelInfo"]["metrics"]["F1_WEIGHTED"]["test"]

0.7287325951256631

## Using the model for prediction

After training, the model is ready to classify unclassified data. 

In [ ]:
model.predict_mutate(
    G,
    mutateProperty="predictedClass",
    modelName="cora-pipeline-model",
    predictedProbabilityProperty="predictedProbabilities",
    targetNodeLabels=["UnclassifiedPaper"],
)

predicted = gds.graph.streamNodeProperty(G, "predictedClass", ["UnclassifiedPaper"])

In [ ]:
predicted

,nodeId,propertyValue
0,0,0
1,1,5
2,2,2
3,3,2
4,4,3
5,5,5
6,6,6
7,7,0
8,8,0
9,9,4


In [ ]:
# Retrieve node information from Neo4j using the node IDs from the prediction result
nodes = gds.util.asNodes(predicted.nodeId.to_list())

# Create a new DataFrame containing node IDs along with node properties
nodes_df = pd.DataFrame([(node.id, node["subject"]) for node in nodes], columns=["nodeId", "subject"])

# Merge with the prediction result on node IDs, to check the predicted value
# against the original subject
predicted.merge(nodes_df, on="nodeId")

,nodeId,propertyValue,subject
0,0,0,0
1,1,5,1
2,2,2,2
3,3,2,2
4,4,3,3
5,5,5,3
6,6,6,4
7,7,0,0
8,8,0,0
9,9,4,4


## Writing result back to Neo4j

Having the predicted class written back to the graph, we can now write them back to the Neo4j database.

In [ ]:
gds.graph.nodeProperties.write(
    G,
    node_properties=["predictedClass"],
    node_labels=["UnclassifiedPaper"],
)

writeMillis                        14
graphName                  cora-graph
nodeProperties       [predictedClass]
propertiesWritten                  10
Name: 0, dtype: object

## Cleanup

When the graph, the model and the pipeline are no longer needed, they should be dropped to free up memory. This only needs to be done if the Neo4j or AuraDS instance is not restarted, since a restart would clean up all the in-memory content anyway.

In [ ]:
model.drop()
model_fastrp.drop()
node_pipeline.drop()
node_pipeline_fastrp.drop()

G.drop()

The Neo4j database instead needs to be cleaned up explicitly if no longer useful:

In [ ]:
gds.run_cypher("MATCH (n) WHERE n:Paper OR n:UnclassifiedPaper DETACH DELETE n")

It is good practice to close the client as well:

In [ ]:
gds.close()